## Notebook rilut

In [1]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from operator import itemgetter
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

def feature_importances(Xdf, rfc):
    pair=zip(Xdf.columns, rfc.feature_importances_)
    t1=[i for i in pair]; t1
    return sorted(t1, key=itemgetter(1))

In [19]:
# Pakai make_datasets_train_only. Buat sendiri juga boleh

df=pd.read_csv('train_gue.csv')

if 'deviceID' in df.columns:
    del df['deviceID']

df.head()

,customerID,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,...,total_order,budget,articleID_prob,colorCode_prob,customerID_prob,sizeCode_prob,paymentMethod_prob,after_voucher,order_order,choice_order
0,c1000001,a1001465,2014-01-03,i1003077,3999,I,17.0,1,27.984375,27.984375,...,27.984375,69.625,0.509766,0.510254,0.478516,0.374268,0.560547,27.984375,0,0
1,c1000001,a1004977,2014-01-07,i1003082,1999,I,17.0,1,25.984375,25.984375,...,25.984375,69.625,0.427734,0.522461,0.478516,0.374268,0.560547,25.984375,1,0
2,c1000001,a1012517,2014-01-15,i1001229,1993,38,4.0,1,40.000000,40.000000,...,40.000000,69.625,0.473145,0.471680,0.478516,0.535645,0.560547,40.000000,2,0
3,c1000001,a1013536,2014-01-16,i1002531,1995,38,15.0,1,60.000000,60.000000,...,120.000000,69.625,0.502930,0.541992,0.478516,0.535645,0.560547,120.000000,3,0
4,c1000001,a1013536,2014-01-16,i1002531,1999,40,15.0,1,60.000000,60.000000,...,120.000000,69.625,0.502930,0.522461,0.478516,0.529297,0.560547,120.000000,4,1


In [22]:
column='ac'
columns=['articleID','colorCode']
df2 = df[['articleID','colorCode','returnQuantity','quantity']]
df_return_probability = df2.groupby(columns).sum()
df_return_probability[ 'ac' + '_prob' ]  = df_return_probability.returnQuantity / df_return_probability.quantity
ac_prob_dict=df_return_probability.ac_prob.to_dict()
df['ac_prob']=df[['articleID','colorCode']].apply(tuple, axis=1).apply(ac_prob_dict.get).replace(np.nan, 0.5).replace(np.inf, 0.5)

In [23]:
column='as'
columns=['articleID','sizeCode']
df2 = df[['articleID','sizeCode','returnQuantity','quantity']]
df_return_probability = df2.groupby(columns).sum()
df_return_probability[ 'as' + '_prob' ]  = df_return_probability.returnQuantity / df_return_probability.quantity
as_prob_dict=df_return_probability.as_prob.to_dict()
df['as_prob']=df[['articleID','colorCode']].apply(tuple, axis=1).apply(ac_prob_dict.get).replace(np.nan, 0.5).replace(np.inf, 0.5)

In [6]:
# Tambah metodemu di sini atau terserah di manapun!

# metodemu

In [25]:
#Konversi kategori/object ke numerik
print "Konversi kategori/object ke angka (ordinal):"

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns

for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
if 'orderID' in df.columns:
    del df['orderID']

Konversi kategori/object ke angka (ordinal):
customerID
orderID
orderDate
articleID
sizeCode
voucherID
paymentMethod
mmdd


In [27]:
# Presisi float dan int ga usah tinggi-tinggi amat

float_64_columns = df.loc[:, df.dtypes == np.float64].columns
for col in float_64_columns:
    df[col] = df[col].astype(np.float32)

int_64_columns = df.loc[:, df.dtypes == np.int64].columns
for col in int_64_columns:
    df[col] = df[col].astype(np.int32)
    
df.price=df.price.astype(np.int8)
#df.budget=df.budget.astype(np.int8)
#df.after_voucher=df.after_voucher.astype(np.int8)

# Fill NaN: Cara impute terserah!
# Aku sendiri nggak drop row, karena takut probabilitasnya jadi berubah. Tapi bebas kok selama hasilnya lebih baik

df.rrp.fillna(df.rrp.median(), inplace=True)
df['voucherID'].fillna(-1, inplace=True)
df.productGroup.fillna(-1, inplace=True)
df.rrp=df.rrp.astype(np.int8)


Xdf=df.drop([label, "colorCode"], axis=1)

X=Xdf.values
y=df.returnQuantity.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [28]:
model=LinearRegression(n_jobs=-1, normalize=True)
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)

In [29]:
y_pred=model.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[117171  50569     34      0      0      0]
 [ 31904 148465     11      0      0      0]
 [     5    564     17      0      0      0]
 [     0     15     10      0      0      0]
 [     0      0      5      0      0      0]
 [     0      0      5      0      0      0]]
83191


# 83191

In [40]:
rfr=RandomForestRegressor(20, n_jobs=3)
rfr.fit(X_train, y_train)
y_pred_rfr=rfr.predict(X_test)
y_pred_rfr=np.array([round(yp) for yp in y_pred_rfr], dtype=np.int8)
y_pred_rfr[y_pred_rfr<0]=0
print confusion_matrix(y_test,y_pred_rfr)
print dmc_error(y_test,y_pred_rfr)

[[113009  54733     32      0      0      0]
 [ 35595 144681     98      4      2      0]
 [    30    349    205      2      0      0]
 [     1     10      9      4      1      0]
 [     1      3      0      0      0      1]
 [     0      2      2      1      0      0]]
90978


In [41]:
feature_importances(Xdf, rfr)

[('voucherAmount', 0.0047844899661131993),
 ('paymentMethod_prob', 0.0067785851992741954),
 ('paymentMethod', 0.0068497985329638647),
 ('productGroup', 0.0077699268148344759),
 ('months', 0.0081673584800115266),
 ('voucherID', 0.010404496257382431),
 ('rrp', 0.01462534207005764),
 ('sizeCode', 0.01851040797543501),
 ('sizeCode_prob', 0.018637639891351698),
 ('price', 0.023485303450909777),
 ('choice_order', 0.024089116825616042),
 ('quantity', 0.026324213662831043),
 ('after_voucher', 0.031319850491679693),
 ('total_order', 0.031492318102909679),
 ('articleID', 0.036097754602072415),
 ('colorCode_prob', 0.036726653983334995),
 ('mmdd', 0.037155770396917628),
 ('orderDate', 0.037822760120176428),
 ('articleID_prob', 0.038965204935634566),
 ('ac_prob', 0.041011905322619648),
 ('order_order', 0.043129106127019548),
 ('as_prob', 0.048022345383127302),
 ('customerID', 0.049500284371058023),
 ('budget', 0.053221500172878466),
 ('customerID_prob', 0.34510786686379075)]

In [38]:
y_pred_ens=(1*rfr.predict(X_test) + 9*model.predict(X_test))/10
y_pred_ens=np.array([round(yp) for yp in y_pred_ens], dtype=np.int8)
y_pred_ens[y_pred_ens<0]=0
print confusion_matrix(y_test,y_pred_ens)
print dmc_error(y_test,y_pred_ens)

[[117475  50279     20      0      0      0]
 [ 31835 148530     15      0      0      0]
 [     8    544     34      0      0      0]
 [     0     14     11      0      0      0]
 [     0      2      2      1      0      0]
 [     0      0      5      0      0      0]]
82794


```

```